In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Reading

In [49]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [50]:
Data=pd.read_excel('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/new data/hasoc_2020_de_train_new.xlsx',sep='\t')
Data.head()

,tweet_id,text,task1,task2,ID
0,1133388798925189122,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NOT,NONE,hasoc_2020_de_2684
1,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen ihrer fehl...,NOT,NONE,hasoc_2020_de_1042
2,1128897106171842560,"@ruhrbahn jeden Morgen eine neue „Fahrzeugstörung“, ihr seid einfach nur zum Kotzen",NOT,NONE,hasoc_2020_de_774
3,1123576753199484928,"@Junge_Freiheit Die Inkas hatten sich schon dämlich angestellt, bei den spanischen Eindringlinge...",NOT,NONE,hasoc_2020_de_559
4,1128743783393312768,RT @technosteron: leute die 'boar' schreiben lassen sich bestimmt auch von ihren analphabetische...,HOF,PRFN,hasoc_2020_de_1969


# Data Preprocessing
Stop Word Removal

In [51]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('german')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
Data['text'] = Data['text'].apply(lambda x: remove_sw(x))
Data[['text','task2']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,task2
0,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NONE
1,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden ...,NONE
2,"@ruhrbahn Morgen neue „Fahrzeugstörung“, seid einfach Kotzen",NONE
3,"@Junge_Freiheit Die Inkas schon dämlich angestellt, spanischen Eindringlingen, Deutsche toppt",NONE
4,RT @technosteron: leute 'boar' schreiben lassen bestimmt analphabetischen Vater arsch ficken,PRFN


Removing Emoji

In [52]:
#rem0ving emoji

import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)

Data['text']=Data['text'].apply(lambda x:deEmojify(x))
Data[['text','task2']].head()

,text,task2
0,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NONE
1,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden ...,NONE
2,"@ruhrbahn Morgen neue „Fahrzeugstörung“, seid einfach Kotzen",NONE
3,"@Junge_Freiheit Die Inkas schon dämlich angestellt, spanischen Eindringlingen, Deutsche toppt",NONE
4,RT @technosteron: leute 'boar' schreiben lassen bestimmt analphabetischen Vater arsch ficken,PRFN


Removing URL

In [53]:
# remove URL
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)

Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data[['text']].head()

Data['text']=Data['text'].str.replace(r'\d+','')
Data[['text','task2']].head()

,text,task2
0,Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch ...,NONE
1,RT @NDRinfo: Die deutsche Klimaaktivistin Luisa Neubauer wirft Kanzlerin Merkel wegen fehlenden ...,NONE
2,"@ruhrbahn Morgen neue „Fahrzeugstörung“, seid einfach Kotzen",NONE
3,"@Junge_Freiheit Die Inkas schon dämlich angestellt, spanischen Eindringlingen, Deutsche toppt",NONE
4,RT @technosteron: leute 'boar' schreiben lassen bestimmt analphabetischen Vater arsch ficken,PRFN


# Tokenization, Encoding, padding after preprocessing

In [54]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [55]:
tok1.fit_on_texts(Data['text'])
words=len(tok1.word_counts)
words

9265

In [56]:
word_list=[tok1.word_index]
#word_list

In [57]:
encoded=tok1.texts_to_sequences(Data['text'])
#print(Data['text'][0])
encoded[1211]

[1, 5723, 550, 417, 319, 19, 203, 185, 495, 45, 485, 110, 105, 1844, 5724]

In [58]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
#padd[0:12]

# Label Encoding

In [59]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['task2'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(Data['task2'])
train_label=to_categorical(np.asarray(train_labelEncode))

['HATE' 'NONE' 'OFFN' 'PRFN']


# Importing libraries for Model creation

In [60]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

#CNN model

In [87]:
model = Sequential()
model.add(Embedding(input_dim = words + 1, output_dim = 100,input_length=max_length))
model.add(Conv1D(filters =512,kernel_size=2, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 100)           926600    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 19, 512)           102912    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 512)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               393344    
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 516       
Total params: 1,423,372
Trainable params: 1,423,372
Non-trainable params: 0
____________________________________________

# Model Compilation

In [88]:
## compile

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Model training

In [99]:
model.fit(padd,train_label,epochs=15,verbose=1,batch_size= 32)

Epoch 1/15
75/75 [==============================] - 1s 11ms/step - loss: 3.4331e-05 - acc: 1.0000
Epoch 2/15
75/75 [==============================] - 1s 10ms/step - loss: 3.1631e-05 - acc: 1.0000
Epoch 3/15
75/75 [==============================] - 1s 11ms/step - loss: 2.9285e-05 - acc: 1.0000
Epoch 4/15
75/75 [==============================] - 1s 10ms/step - loss: 2.7041e-05 - acc: 1.0000
Epoch 5/15
75/75 [==============================] - 1s 11ms/step - loss: 2.5086e-05 - acc: 1.0000
Epoch 6/15
75/75 [==============================] - 1s 11ms/step - loss: 2.3321e-05 - acc: 1.0000
Epoch 7/15
75/75 [==============================] - 1s 11ms/step - loss: 2.1758e-05 - acc: 1.0000
Epoch 8/15
75/75 [==============================] - 1s 11ms/step - loss: 2.0261e-05 - acc: 1.0000
Epoch 9/15
75/75 [==============================] - 1s 10ms/step - loss: 1.8908e-05 - acc: 1.0000
Epoch 10/15
75/75 [==============================] - 1s 10ms/step - loss: 1.7671e-05 - acc: 1.0000
Epoch 11/15
75/75 [

# Test data reading

In [100]:
Test =pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/test data/german_test_1509.csv')
Test.dropna(inplace = True)
Test.head()

,tweet_id,text,task1,task2,ID
0,1129095874242650112,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT,NONE,hasoc_2020_de_1053
1,1129004308396236800,"RT @ibikus31: Es wird spekuliert, ob Merkel ein Amt in Brüssel übernimmt.\nWäre es so, wie schät...",NOT,NONE,hasoc_2020_de_671
2,1130896929355907080,Hat #Hitler wirklich den Krieg in der Wüste verloren? Nach der #Welt Schlagzeile hat die #Tagess...,NOT,NONE,hasoc_2020_de_2977
3,1132251534329307136,"RT @Beatrix_vStorch: #May tritt in UK unter Tränen zurück. Wenn #Merkel zurücktritt, dann auch u...",NOT,NONE,hasoc_2020_de_1746
4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen wieder ab in ihre warmen Überwinterungsquartiere. 😅😂,NOT,NONE,hasoc_2020_de_2416


# Test data preprocessing

In [101]:
## doing preprocessing as above

Test['text'] = Test['text'].apply(lambda x: remove_sw(x))
Test['text']=Test['text'].apply(lambda x:deEmojify(x))
Test['text']=Test['text'].apply(lambda x: remove_URL(x))
Test['text']=Test['text'].str.replace(r'\d+','')
t= Test['text']

# Test data encoding, padding

In [102]:
encoded1 =tok1.texts_to_sequences(t)
print(t[0])
encoded1[0]

@derCarsti Boykottieren hört besser an. 


[45, 99, 83]

In [103]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([45, 99, 83,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

# Test label encoding

In [104]:
test_labelEncode=labelEncode.transform(Test['task2'])
test_label=to_categorical(np.asarray(test_labelEncode))

# Test data Model prediction

In [105]:
test_predictions=model.predict(padded)
test_predictions2=np.argmax(test_predictions,axis=1)

# Test data Classification report

In [106]:
#Accuracy 
from sklearn.metrics import classification_report
print(classification_report(test_labelEncode,test_predictions2))

              precision    recall  f1-score   support

           0       0.29      0.25      0.27        24
           1       0.85      0.89      0.87       378
           2       0.17      0.06      0.08        36
           3       0.64      0.74      0.69        88

    accuracy                           0.78       526
   macro avg       0.49      0.48      0.48       526
weighted avg       0.75      0.78      0.76       526



# Adding predicted lable as a colum to dataFrame 'Test'

In [107]:
Test['Predictionlabel']=labelEncode.inverse_transform(test_predictions2)
Test.head()

,tweet_id,text,task1,task2,ID,Predictionlabel
0,1129095874242650112,@derCarsti Boykottieren hört besser an.,NOT,NONE,hasoc_2020_de_1053,NONE
1,1129004308396236800,"RT @ibikus: Es spekuliert, Merkel Amt Brüssel übernimmt. Wäre so, schätzen Sie Zukunft Mitglieds...",NOT,NONE,hasoc_2020_de_671,NONE
2,1130896929355907080,Hat #Hitler wirklich Krieg Wüste verloren? Nach #Welt Schlagzeile #Tagesschau reagiert.,NOT,NONE,hasoc_2020_de_2977,NONE
3,1132251534329307136,"RT @Beatrix_vStorch: #May tritt UK Tränen zurück. Wenn #Merkel zurücktritt, Tränen – Freudenträn...",NOT,NONE,hasoc_2020_de_1746,NONE
4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen ab warmen Überwinterungsquartiere.,NOT,NONE,hasoc_2020_de_2416,NONE


# Generating CSV file

In [109]:
GermanTask2File=Test[['tweet_id','Predictionlabel','ID']]
GermanTask2File.head()
GermanTask2File.to_csv('submission_DE_B.csv',index=False)